In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [6]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"class":'a-size-large product-title-word-break'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-offscreen'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count


In [28]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.3 Safari/605.1.15', 'Accept-Language': 'en-US, en;q=0.5'})
    
    pages= np.arange(1,21)
    for page in pages:
        
        # The webpage URL
        URL = "https://www.amazon.in/s?k=bags&page="+str(page)+"&qid=1679393196&ref=sr_pg_"+str(page)
        # HTTP Request
        webpage = requests.get(URL, headers=HEADERS)

        # Soup Object containing all data
        soup = BeautifulSoup(webpage.content, "html.parser")

        # Fetch links as List of Tag Objects
        links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

        # Store the links
        links_list = []

        Product_Link=[]
        Product_Name=[]
        Product_price=[]
        Rating=[]
        Number_of_reviews=[]


        # Loop for extracting links from Tag Objects
        for link in links:
                links_list.append(link.get('href'))

        # Loop for extracting product details from each link 
        for link in links_list:
            dlink="https://www.amazon.com" + link
            new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

            new_soup = BeautifulSoup(new_webpage.content, "html.parser")

            Product_Link.append(dlink)
            Product_Name.append(get_title(new_soup))
            Product_price.append(get_price(new_soup))
            Rating.append(get_rating(new_soup))
            Number_of_reviews.append(get_review_count(new_soup))

        #Exporting to CSV
        amazon_df = pd.DataFrame({"Product Link":Product_Link,"Product Name":Product_Name,"Product Price":Product_price,"Rating":Rating,"Number of reviews":Number_of_reviews})
        amazon_df.to_csv("Amazon_data(task1).csv")